In [2]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [7]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


## Wczytywanie danych

In [8]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

# Dummy model


In [10]:
df.select_dtypes(np.number).columns   # sprawdź które kolumny są liczbowe

Index(['price_value', 'car_id'], dtype='object')

In [13]:
feats = ['car_id'] # lista cech
X = df[ feats ].values  # lub X=df[['car_id]], podwójny nawias
y = df['price_value'].values # wektor

model = DummyRegressor() # tworzymy nowy model
model.fit(X, y) # trenowanie modelu, model dummy zwraca średnią z price_value
y_pred = model.predict(X) # prognozowanie po X

mae(y, y_pred)

39465.934630440985

In [15]:
[x for x in df.columns if 'price' in x] # pętla przegląda wszystkie kolumny i sprawdza czy w nazwie jest price

['price_currency', 'price_details', 'price_value']

In [17]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [20]:
df['price_currency'].value_counts(normalize=True) * 100

PLN    100.0
Name: price_currency, dtype: float64

In [19]:
df = df[ df['price_currency'] != 'EUR']
df.shape

(106290, 155)

# Features


In [23]:
df['param_color'].factorize()[0]

array([-1, -1, -1, ..., -1, -1, -1])

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:          # jeśli już kolumna posiada faktoryzowaną kolumnę
    df[feat] = factorized_values  # to nadpisz ją tymi samymi wartościami
  else:                           # a jeśli nie posiada
    df[feat + SUFFIX_CAT] = factorized_values # to dodaj kolumnę

In [28]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x] # do zmiennej wszystkie kolumny z '__cat'
cat_feats = [x for x in cat_feats if 'price' not in x] # usuwamy wszystkie zawierające 'price'
len(cat_feats)

151

In [29]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368324

In [34]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2533 ± 0.0040,param_napęd__cat
0.2008 ± 0.0047,param_faktura-vat__cat
0.1943 ± 0.0088,param_stan__cat
0.1423 ± 0.0086,param_rok-produkcji__cat
0.0629 ± 0.0047,param_moc__cat
0.0424 ± 0.0014,feature_kamera-cofania__cat
0.0412 ± 0.0008,param_skrzynia-biegów__cat
0.0286 ± 0.0037,param_marka-pojazdu__cat
0.0191 ± 0.0022,param_pojemność-skokowa__cat
0.0163 ± 0.0005,feature_bluetooth__cat
